# NUS GenAI Capstone Project

## Objective:
Submit a fully integrated prototype that supports the following:
1. Conversational interface with limited memory
2. Document-based Question Answering using RAG
3. Text-to-image generation with prompt engineering
4. Multi-agent task handling using a controller (Weather, SQL, Recommender)
5. Final technical report on system design, debugging, and improvements

## Setup

In [1]:
# Import libraries

# === Core Python Libraries ===
import os
import replicate
import sqlite3
import requests
from IPython.display import display, Markdown
from datetime import datetime, timedelta
from typing import TypedDict, Annotated

# === LangChain 1.0 - Agent Framework ===
from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

# === LangGraph 1.0 - State Management & Checkpointing ===
from langgraph.checkpoint.memory import MemorySaver

# === Document Processing & Vector Storage ===
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

# === Hybrid RAG - BM25 and Ensemble Retrievers ===
from langchain_community.retrievers import BM25Retriever
from langchain_classic.retrievers import EnsembleRetriever, ContextualCompressionRetriever

# === Reranking - Jina AI Cross-Encoder ===
from langchain_community.document_compressors import JinaRerank

In [2]:
# Retrieve API keys from environment variables
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
REPLICATE_API_TOKEN = os.environ.get("REPLICATE_API_TOKEN")
WEATHER_API_KEY = os.environ.get("WEATHER_API_KEY")
JINA_API_KEY = os.environ.get("JINA_API_KEY")

In [4]:
# Prepare Hybrid RAG system for Singapore venue policies
# Load venue policy documents for Marina Bay Sands, Gardens by the Bay, Esplanade, and SG regulations

venue_policy_files = [
    'MBS-Event-Policy.pdf',
    'GBTB-Venue-Guide.pdf',
    'Esplanade-Manual.pdf',
    'SG-Event-Regulations.pdf'
]

# Load all venue policy documents
all_documents = []
for filepath in venue_policy_files:
    loader = PyMuPDFLoader(filepath, mode="single")
    docs = loader.load()
    if docs:
        all_documents.extend(docs)
        print(f"✅ Loaded {len(docs)} pages from {filepath}")
    else:
        print(f"⚠️ No content extracted from {filepath}")

if not all_documents:
    raise ValueError("No documents loaded. Check PDF files.")

print(f"\n📄 Total pages loaded: {len(all_documents)}")

# Split into chunks optimized for policy documents
# Using smaller chunk_size (1000 vs 2000) for better granularity with policy sections
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(all_documents)
if not chunks:
    raise ValueError("No chunks produced. Check document parsing.")
print(f"📋 Split into {len(chunks)} chunks (1000 char size, 200 overlap)")

# Create embeddings
embedding_model = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model="text-embedding-3-small")
print("✅ Embedding model initialized")

# Create semantic retriever (dense vector search)
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="./venue_policies_chroma_db"
)
semantic_retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})
print("✅ Semantic retriever created (Chroma + OpenAI embeddings)")

# Create BM25 retriever (keyword/sparse search)
bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 5
print("✅ BM25 retriever created (keyword-based)")

# Combine retrievers with Reciprocal Rank Fusion (RRF)
hybrid_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, semantic_retriever],
    weights=[0.4, 0.6]  # 40% keyword (BM25), 60% semantic
)
print("✅ Hybrid retriever created (BM25 + Semantic with RRF)")
print("   Weights: 40% keyword, 60% semantic")

# Add Jina AI reranker on top of hybrid retrieval (3-stage retrieval)
compressor = JinaRerank(
    model="jina-reranker-v2-base-multilingual",  # Jina Reranker v2 (v3 may not be in API yet)
    top_n=3,  # Return top 3 after reranking
    jina_api_key=JINA_API_KEY
)

reranking_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=hybrid_retriever
)
print("✅ Reranking retriever created (Jina Reranker v2)")
print("   3-stage pipeline: BM25 → Semantic → Jina cross-encoder rerank")

print("\n🎯 Complete retrieval system ready:")
print("   Stage 1: BM25 keyword search (exact matches)")
print("   Stage 2: Semantic vector search (conceptual)")  
print("   Stage 3: Jina AI cross-encoder reranking (relevance)")
print("\n✨ Optimized for venue policy queries with proper nouns, codes, and concepts!")

✅ Loaded 1 pages from MBS-Event-Policy.pdf
✅ Loaded 1 pages from GBTB-Venue-Guide.pdf
✅ Loaded 1 pages from Esplanade-Manual.pdf
✅ Loaded 1 pages from SG-Event-Regulations.pdf

📄 Total pages loaded: 4
📋 Split into 103 chunks (1000 char size, 200 overlap)
✅ Embedding model initialized
✅ Semantic retriever created (Chroma + OpenAI embeddings)
✅ BM25 retriever created (keyword-based)
✅ Hybrid retriever created (BM25 + Semantic with RRF)
   Weights: 40% keyword, 60% semantic
✅ Reranking retriever created (Jina Reranker v2)
   3-stage pipeline: BM25 → Semantic → Jina cross-encoder rerank

🎯 Complete retrieval system ready:
   Stage 1: BM25 keyword search (exact matches)
   Stage 2: Semantic vector search (conceptual)
   Stage 3: Jina AI cross-encoder reranking (relevance)

✨ Optimized for venue policy queries with proper nouns, codes, and concepts!


In [10]:
# Create events database
def setup_database():
    """Initialize the events database with sample data."""
    conn = sqlite3.connect('events.db')
    c = conn.cursor()

    # Create table if not exists
    c.execute('''
        CREATE TABLE IF NOT EXISTS events (
            id INTEGER PRIMARY KEY,
            name TEXT,
            type TEXT,  -- 'indoor' or 'outdoor'
            description TEXT,
            location TEXT,
            country TEXT,
            date TEXT
        )
    ''')

    today = datetime.now().date()
    def iso(days=0): return (today + timedelta(days=days)).isoformat()

    # Synthetic event data
    events = [
        ("Marina Bay Food Festival", "outdoor", "A celebration of local and international cuisine", "Marina Bay, Singapore", "Singapore", iso(0)),
        ("Orchard Mall Art Fair", "indoor", "Pop-up art and design exhibition", "ION Orchard, Singapore", "Singapore", iso(0)),
        ("Mumbai Music Street", "outdoor", "Live indie music performances", "Marine Drive, Mumbai", "India", iso(0)),
        ("Delhi Book Conclave", "indoor", "Writers and readers meet-up", "Pragati Maidan, New Delhi", "India", iso(0)),
        ("Bangkok Street Carnival", "outdoor", "Street performances and food stalls", "Siam Square, Bangkok", "Thailand", iso(0)),
        ("Thai Craft Showcase", "indoor", "Traditional Thai crafts and art", "Bangkok Art Center, Bangkok", "Thailand", iso(0)),
        ("Penang Heritage Walk", "outdoor", "Tour of George Town’s historic district", "George Town, Penang", "Malaysia", iso(0)),
        ("KL Coffee Expo", "indoor", "Coffee tasting and workshops", "KL Convention Centre, Kuala Lumpur", "Malaysia", iso(0)),
        ("Jakarta Film Screening", "indoor", "Indie film premieres", "Cinema XXI, Jakarta", "Indonesia", iso(0)),
        ("Bali Sunset Beach Fest", "outdoor", "Beach music and food event", "Canggu, Bali", "Indonesia", iso(0)),
        ("Hanoi Street Parade", "outdoor", "Music and cultural performances", "Old Quarter, Hanoi", "Vietnam", iso(0)),
        ("Hanoi Art Studio", "indoor", "Local artist exhibition", "French Quarter, Hanoi", "Vietnam", iso(0)),
        ("Manila Food Market", "outdoor", "Filipino cuisine and music", "Intramuros, Manila", "Philippines", iso(0)),
        ("Manila Tech Expo", "indoor", "Startup and innovation exhibition", "SMX Convention Center, Manila", "Philippines", iso(0)),
        ("Singapore Jazz Night", "indoor", "Regional jazz bands live", "Esplanade, Singapore", "Singapore", iso(1)),
        ("Singapore Botanic Fair", "outdoor", "Flower and plant exhibition", "Singapore Botanic Gardens, Singapore", "Singapore", iso(1)),
        ("Chennai Dance Gala", "indoor", "Classical Bharatanatyam showcase", "Music Academy, Chennai", "India", iso(1)),
        ("Goa Beach Fest", "outdoor", "Open-air music by the sea", "Baga Beach, Goa", "India", iso(1)),
        ("Bangkok Food Carnival", "outdoor", "Street food extravaganza", "Chatuchak Market, Bangkok", "Thailand", iso(1)),
        ("Bangkok Innovation Hub", "indoor", "Tech startups and product demos", "Siam Discovery, Bangkok", "Thailand", iso(1)),
    ]

    # Insert data safely
    c.executemany('''
        INSERT OR IGNORE INTO events (name, type, description, location, country, date)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', events)

    conn.commit()
    print("✅ Database setup completed successfully. Events table is ready.")
    conn.close()

# Run setup
setup_database()

✅ Database setup completed successfully. Events table is ready.


## Implementation

In [11]:
@tool
def retrieve_venue_policies(query: str) -> str:
    """
    Retrieve venue policies, restrictions, and requirements from Singapore event venues.
    Uses 3-stage retrieval: BM25 + semantic + Jina Reranker v2 for maximum relevance.
    
    Covers:
    - Photography/equipment restrictions (tripods, drones, professional gear)
    - Sound level limits and amplified music regulations  
    - Capacity limits and safety requirements
    - Accessibility features (wheelchair access, assisted listening)
    - Insurance requirements and liability coverage
    - MRT access, postal codes, and parking information
    - Technical specifications (stage dimensions, lighting, sound systems)
    
    Args:
        query: Question about venue policies (e.g., "tripod policy at Marina Bay Sands")
    
    Returns:
        Relevant policy excerpts with source attribution (venue name and page number)
    """
    # Use reranking retriever for best results
    retrieved_docs = reranking_retriever.invoke(query)
    
    # Format with source attribution for transparency
    formatted = []
    for doc in retrieved_docs:
        source = doc.metadata.get('source', 'Unknown')
        page = doc.metadata.get('page', '?')
        # Extract just the filename without path
        source_name = source.split('\\')[-1].split('/')[-1].replace('.pdf', '')
        formatted.append(f"[{source_name}, p.{page}]\n{doc.page_content}\n")
    
    return "\n".join(formatted)

# Store pending image requests for approval
pending_image_requests = {}

@tool
def request_image_generation(prompt: str, seed: int = 42, steps: int = 30) -> str:
    """
    Request to generate an image using Replicate API.
    This will ask for user approval before actually generating the image (costs money).
    
    Args:
        prompt: Description of the image to generate
        seed: Random seed for reproducibility (default: 42)
        steps: Number of generation steps (default: 30)
    
    Returns:
        A message indicating approval is needed
    """
    import uuid
    request_id = str(uuid.uuid4())
    pending_image_requests[request_id] = {
        "prompt": prompt,
        "seed": seed,
        "steps": steps
    }
    return f"🖼️ Image generation requested for: '{prompt}'\n\n⚠️ This will cost money via Replicate API. Please approve by calling 'approve_image_generation' with request_id: {request_id}"

@tool
def approve_image_generation(request_id: str) -> str:
    """
    Approve and execute a pending image generation request.
    
    Args:
        request_id: The ID of the pending image request to approve
    
    Returns:
        URL of the generated image or error message
    """
    if request_id not in pending_image_requests:
        return "⚠️ Invalid or expired request ID. No pending image generation found."
    
    request = pending_image_requests.pop(request_id)
    prompt = request["prompt"]
    seed = request["seed"]
    steps = request["steps"]
    
    try:
        output = replicate.run(
            "stability-ai/stable-diffusion-3.5-medium",
            input={"prompt": prompt, "seed": seed, "steps": steps}
        )

        # Handle unexpected response formats
        if isinstance(output, list):
            return output[0] if output else "⚠️ No image generated."
        elif hasattr(output, "url"):
            return output.url
        else:
            return str(output)
    except replicate.exceptions.ModelError as e:
        return f"⚠️ Image generation model error: {e}"
    except replicate.exceptions.ReplicateError as e:
        return f"⚠️ Replicate API error: {e}"
    except Exception as e:
        return f"⚠️ Unexpected image generation error: {e}"

@tool
def get_current_date() -> str:
    """
    Returns today's date in ISO format (YYYY-MM-DD).
    Use this tool when you need to know the current date for querying events or making date-based recommendations.
    """
    return datetime.now().date().isoformat()

In [12]:
@tool
def get_weather(location: str = 'Singapore') -> str:
    """
    Retrieve real-time weather data via the WeatherAPI.
    Takes a location as input and returns weather information including temperature and conditions.
    """
    url = "http://api.weatherapi.com/v1/current.json"
    params = {"key": WEATHER_API_KEY, "q": location, "aqi": "no"}
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        # Extract relevant weather information
        location_name = data['location']['name']
        country = data['location']['country']
        temp_c = data['current']['temp_c']
        condition = data['current']['condition']['text']
        
        return f"Weather in {location_name}, {country}: {temp_c}°C, {condition}"
    except requests.exceptions.Timeout:
        return f"⚠️ Weather API request timed out for {location}."
    except requests.exceptions.ConnectionError:
        return f"⚠️ Network connection failed while fetching weather for {location}."
    except requests.exceptions.HTTPError as e:
        return f"⚠️ Weather API HTTP error: {e}"
    except KeyError:
        return f"⚠️ Unexpected weather data format for {location}."
    except Exception as e:
        return f"⚠️ Unexpected weather retrieval error: {e}"

In [13]:
@tool
def get_events(date: str, event_type: str | None = None, country: str = 'Singapore') -> str:
    """
    Retrieves event data by querying the SQLite database for events on a given date.
    Optionally filters by event_type (indoor/outdoor) and country (default: Singapore).
    Returns a formatted string of matching events.
    """
    conn = sqlite3.connect('events.db')
    c = conn.cursor()
    
    if event_type:
        c.execute('SELECT * FROM events WHERE date=? AND type=? AND country=?', (date, event_type, country))
    else:
        c.execute('SELECT * FROM events WHERE date=? AND country=?', (date, country))
    
    events = c.fetchall()
    conn.close()
    if not events:
        return f"No events found in {country} on {date}" + (f" ({event_type} type)" if event_type else " (all types)")
    
    # Format events nicely
    formatted_events = []
    for event in events:
        event_id, name, etype, desc, location, ecountry, edate = event
        formatted_events.append(
            f"- {name} ({etype}): {desc}. Location: {location}. Date: {edate}"
        )
    
    return "\n".join(formatted_events)

In [14]:
# Recommendation Tool - Simple LLM Chain

# Create prompt template for recommendations
recommendation_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful event recommender. Consider the weather conditions and suggest suitable events. 
    For outdoor events, consider the temperature and weather conditions. Be specific about why you recommend certain events over others. 
    Keep your response concise but informative. 
    If event data is unavailable, politely request the user for additional event-related information.
    If weather data is unavailable, provide a balanced mix of indoor and outdoor suggestions."""),
    ("user", "{weather_and_event_data}")
])

# Create a simple chain: prompt → LLM
recommendation_chain = recommendation_prompt | ChatOpenAI(model="gpt-4o", api_key=OPENAI_API_KEY)

@tool
def recommend_events(weather_and_event_data: str) -> str:
    """
    Synthesize weather and event data into context-aware event recommendations.
    Takes combined weather and event information as input and returns personalized suggestions.
    
    Args:
        weather_and_event_data: Combined string containing weather conditions and available events
    
    Returns:
        A concise, personalized recommendation based on weather and events
    """
    try:
        # Invoke the chain with the input data
        result = recommendation_chain.invoke({"weather_and_event_data": weather_and_event_data})
        return result.content
    except Exception as e:
        return f"⚠️ Recommendation failed: {e}"

In [15]:
# Main Agent Setup - LangChain 1.0

# Define tools available to main agent
tools = [
    retrieve_venue_policies,
    request_image_generation,
    approve_image_generation,
    get_current_date,
    get_weather,
    get_events,
    recommend_events
]

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", api_key=OPENAI_API_KEY)

# Create checkpointer for conversation persistence
checkpointer = MemorySaver()

# System prompt for the main agent
system_prompt = """You are an intelligent event planning assistant for Singapore.

**Your Role:**
Help users discover and attend events in Singapore by providing:
1. Event recommendations based on weather and preferences
2. Venue policy information for successful event attendance
3. Image generation for event visualization

**Venue Policy Knowledge (3-Stage Retrieval):**
You have access to detailed policies from major Singapore venues via advanced hybrid RAG + Jina reranking:
- **Marina Bay Sands (MBS)**: Event policies, photography rules, technical specs, postal code 018956
- **Gardens by the Bay (GBTB)**: Sound restrictions, plant protection, photography policies, postal code 018953
- **Esplanade**: Performing arts facilities, technical specifications, accessibility features
- **Singapore Regulations**: PEL licensing, NEA sound limits, SCDF fire safety, insurance requirements

**Retrieval Pipeline:**
The venue policy retrieval uses a 3-stage system for optimal relevance:
1. Stage 1: BM25 keyword search (excels at proper nouns, postal codes, MRT codes)
2. Stage 2: Semantic vector search (understands concepts like "accessibility" or "restrictions")
3. Stage 3: Jina AI cross-encoder reranking (fine-grained relevance scoring)

**Tool Usage Guidelines:**
1. Use 'retrieve_venue_policies' to answer questions about:
   - Photography/equipment restrictions (e.g., "Can I bring a tripod to Marina Bay Sands?")
   - Sound level limits (e.g., "What are the sound restrictions at Gardens by the Bay?")
   - Accessibility features (e.g., "Wheelchair access at Esplanade?")
   - Insurance requirements, capacity limits, MRT access, parking
   - Technical specifications (stage dimensions, lighting, sound systems)

2. Use 'request_image_generation' when the user asks for image generation - this will request approval first.
3. After the user approves, use 'approve_image_generation' with the provided request_id to actually generate the image.
4. Use 'get_current_date' when you need to know today's date for event queries or recommendations.
5. Use 'get_weather' to retrieve weather information for a specific location (default: Singapore).
6. Use 'get_events' to query events from the database by date, type (indoor/outdoor), and country.
7. Use 'recommend_events' to synthesize weather and event data into personalized recommendations.

**Multi-Step Recommendation Workflow:**
When the user asks for event recommendations:
1. First, call 'get_current_date' to know today's date
2. Call 'get_weather' to get weather conditions for the location
3. Call 'get_events' to retrieve available events for the date
4. Finally, call 'recommend_events' with the combined weather and event data to generate personalized suggestions

**Policy Query Best Practices:**
- Always cite specific policy sources when providing venue information (e.g., "[MBS-Event-Policy, p.2]")
- For location-specific queries, use exact terms like "Marina Bay Sands", "postal code 018956", "MRT CE1"
- For conceptual queries, use natural language like "accessibility features" or "sound restrictions"

Be helpful, concise, and informative in your responses. The 3-stage retrieval system provides highly relevant results for both exact-match queries (venue names, codes) and conceptual queries (accessibility, restrictions)."""

# Create the main agent using LangChain 1.0's create_agent
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_prompt,
    checkpointer=checkpointer
)

print("✅ Main agent created successfully with LangChain 1.0")
print("🎯 Agent configured for Singapore event planning with 3-stage retrieval:")
print("   Stage 1: BM25 keyword search")
print("   Stage 2: Semantic vector search")
print("   Stage 3: Jina AI cross-encoder reranking")

✅ Main agent created successfully with LangChain 1.0
🎯 Agent configured for Singapore event planning with 3-stage retrieval:
   Stage 1: BM25 keyword search
   Stage 2: Semantic vector search
   Stage 3: Jina AI cross-encoder reranking


In [16]:
# Chat Interface Functions - Updated for LangChain/LangGraph 1.0

# Thread configuration for conversation persistence
config = {"configurable": {"thread_id": "main_conversation"}}

def chat(user_input: str):
    """Send a message to the agent and display the response."""
    try:
        # Invoke agent with LangGraph 1.0 interface
        result = agent.invoke(
            {"messages": [HumanMessage(content=user_input)]},
            config=config
        )
        
        # Extract the last AI message
        if result and "messages" in result:
            messages = result["messages"]
            # Get the last assistant message
            ai_messages = [msg for msg in messages if isinstance(msg, AIMessage)]
            if ai_messages:
                output = ai_messages[-1].content
                print("AI:")
                display(Markdown(output))
                return {"output": output, "messages": messages}
            else:
                print("⚠️ No response from agent.")
                return {"output": "⚠️ No response generated.", "messages": messages}
        else:
            print("⚠️ Unexpected response format.")
            return {"output": "⚠️ Unexpected response format.", "messages": []}
    
    except Exception as e:
        print(f"⚠️ Unexpected error: {e}")
        import traceback
        traceback.print_exc()
        return {"output": f"⚠️ Unexpected error occurred: {e}"}

def chat_loop():
    """Start an interactive chat session."""
    print("Chat started! Type 'quit' to exit.\n")
    print("💡 Conversation history is preserved across messages within this session.\n")
    while True:
        try:
            user_input = input("You: ").strip()
            if user_input.lower() in ['quit', 'exit', 'q']:
                print("Goodbye!")
                break
            if not user_input:
                continue
            print(f"You: {user_input}")
            chat(user_input)
        except KeyboardInterrupt:
            print("\n🛑 Session interrupted by user.")
            break
        except Exception as e:
            print(f"⚠️ Error during chat loop: {e}")
            continue

def reset_conversation():
    """Reset the conversation by creating a new thread."""
    global config
    import uuid
    config = {"configurable": {"thread_id": str(uuid.uuid4())}}
    print("✅ Conversation history cleared. Starting fresh.")

## Testing

### Hybrid RAG Demo - Venue Policy Queries

This section demonstrates the hybrid RAG system's ability to handle both exact-match queries (proper nouns, codes) and conceptual queries (accessibility, restrictions).

In [5]:
# Test Hybrid RAG with various query types

print("=" * 80)
print("HYBRID RAG TEST - Venue Policy Queries")
print("=" * 80)

# Test 1: Exact-match query (proper noun + specific term)
print("\n1️⃣ EXACT-MATCH QUERY: 'Can I bring tripods to Marina Bay Sands?'")
print("-" * 80)
result1 = hybrid_retriever.invoke("Can I bring tripods to Marina Bay Sands?")
print(f"Retrieved {len(result1)} documents")
print(f"\nTop result preview:")
print(f"Source: {result1[0].metadata.get('source', 'Unknown').split('/')[-1]}")
print(f"Page: {result1[0].metadata.get('page', '?')}")
print(f"Content preview: {result1[0].page_content[:200]}...")

# Test 2: Conceptual query (semantic understanding)
print("\n\n2️⃣ CONCEPTUAL QUERY: 'What are the accessibility features for wheelchair users?'")
print("-" * 80)
result2 = hybrid_retriever.invoke("What are the accessibility features for wheelchair users?")
print(f"Retrieved {len(result2)} documents")
print(f"\nTop result preview:")
print(f"Source: {result2[0].metadata.get('source', 'Unknown').split('/')[-1]}")
print(f"Page: {result2[0].metadata.get('page', '?')}")
print(f"Content preview: {result2[0].page_content[:200]}...")

# Test 3: Hybrid query (location code + conceptual)
print("\n\n3️⃣ HYBRID QUERY: 'Sound restrictions at postal code 018953'")
print("-" * 80)
result3 = hybrid_retriever.invoke("Sound restrictions at postal code 018953")
print(f"Retrieved {len(result3)} documents")
print(f"\nTop result preview:")
print(f"Source: {result3[0].metadata.get('source', 'Unknown').split('/')[-1]}")
print(f"Page: {result3[0].metadata.get('page', '?')}")
print(f"Content preview: {result3[0].page_content[:200]}...")

# Test 4: MRT code query (exact identifier)
print("\n\n4️⃣ LOCATION CODE QUERY: 'Venues near MRT CE1'")
print("-" * 80)
result4 = hybrid_retriever.invoke("Venues near MRT CE1")
print(f"Retrieved {len(result4)} documents")
print(f"\nTop result preview:")
print(f"Source: {result4[0].metadata.get('source', 'Unknown').split('/')[-1]}")
print(f"Page: {result4[0].metadata.get('page', '?')}")
print(f"Content preview: {result4[0].page_content[:200]}...")

print("\n" + "=" * 80)
print("✅ Hybrid RAG system successfully handling diverse query types!")
print("=" * 80)

HYBRID RAG TEST - Venue Policy Queries

1️⃣ EXACT-MATCH QUERY: 'Can I bring tripods to Marina Bay Sands?'
--------------------------------------------------------------------------------
Retrieved 8 documents

Top result preview:
Source: MBS-Event-Policy.pdf
Page: 1
Content preview: Strictly prohibited: No external forklifts or equipment from outside contractors may operate within Marina Bay Sands premises including:
Service ramps
Loading docks
Exhibition halls
Meeting rooms
Ball...


2️⃣ CONCEPTUAL QUERY: 'What are the accessibility features for wheelchair users?'
--------------------------------------------------------------------------------
Retrieved 9 documents

Top result preview:
Source: Esplanade-Manual.pdf
Page: 6
Content preview: Sound recording allowed with prior arrangement
Professional recording services available through Esplanade
Amplified Performances:
Sound levels monitored to protect hearing and venue acoustics
Maximum...


3️⃣ HYBRID QUERY: 'Sound restrictions at pos

KeyboardInterrupt: 

### Evaluation Framework: Hybrid RAG vs Hybrid RAG + Reranking

This section evaluates retrieval quality improvements from adding Cohere reranking to the hybrid RAG system.

In [5]:
# Evaluation Test Dataset - 20 queries across 3 types

test_queries = [
    # === EXACT-MATCH QUERIES (BM25 dominates) ===
    {
        "query": "What are the policies at postal code 018956?",
        "expected_venue": "MBS-Event-Policy",
        "type": "exact_match",
        "description": "Postal code lookup"
    },
    {
        "query": "Venues accessible via MRT CE1",
        "expected_venue": "MBS-Event-Policy",
        "type": "exact_match",
        "description": "MRT code lookup"
    },
    {
        "query": "What venue is located at postal code 018953?",
        "expected_venue": "GBTB-Venue-Guide",
        "type": "exact_match",
        "description": "Postal code lookup"
    },
    {
        "query": "Marina Bay Sands event rules",
        "expected_venue": "MBS-Event-Policy",
        "type": "exact_match",
        "description": "Proper noun match"
    },
    {
        "query": "Esplanade stage specifications",
        "expected_venue": "Esplanade-Manual",
        "type": "exact_match",
        "description": "Proper noun + technical term"
    },
    
    # === SEMANTIC QUERIES (Embeddings dominate) ===
    {
        "query": "What accessibility features are available for disabled visitors?",
        "expected_venue": "Esplanade-Manual",
        "type": "semantic",
        "description": "Conceptual accessibility query"
    },
    {
        "query": "What are the insurance requirements for events?",
        "expected_venue": "SG-Event-Regulations",
        "type": "semantic",
        "description": "General policy concept"
    },
    {
        "query": "Rules about environmental protection at outdoor venues",
        "expected_venue": "GBTB-Venue-Guide",
        "type": "semantic",
        "description": "Environmental concept"
    },
    {
        "query": "Fire safety and emergency procedures",
        "expected_venue": "SG-Event-Regulations",
        "type": "semantic",
        "description": "Safety concept"
    },
    {
        "query": "Lighting and sound system capabilities",
        "expected_venue": "Esplanade-Manual",
        "type": "semantic",
        "description": "Technical capabilities"
    },
    
    # === HYBRID QUERIES (Both BM25 + Semantic needed) ===
    {
        "query": "Can I use tripods at Marina Bay Sands?",
        "expected_venue": "MBS-Event-Policy",
        "type": "hybrid",
        "description": "Proper noun + equipment policy"
    },
    {
        "query": "Sound restrictions at Gardens by the Bay",
        "expected_venue": "GBTB-Venue-Guide",
        "type": "hybrid",
        "description": "Venue name + restriction concept"
    },
    {
        "query": "Wheelchair accessibility at Esplanade",
        "expected_venue": "Esplanade-Manual",
        "type": "hybrid",
        "description": "Venue name + accessibility"
    },
    {
        "query": "Photography equipment restrictions at Marina Bay Sands SkyPark",
        "expected_venue": "MBS-Event-Policy",
        "type": "hybrid",
        "description": "Specific location + policy"
    },
    {
        "query": "What are the sound level limits at postal code 018953?",
        "expected_venue": "GBTB-Venue-Guide",
        "type": "hybrid",
        "description": "Postal code + regulation concept"
    },
    {
        "query": "Stage dimensions and loading capacity at Esplanade Theatre",
        "expected_venue": "Esplanade-Manual",
        "type": "hybrid",
        "description": "Venue + technical specs"
    },
    {
        "query": "Public Entertainment License requirements for Singapore events",
        "expected_venue": "SG-Event-Regulations",
        "type": "hybrid",
        "description": "License type + location"
    },
    {
        "query": "Plant protection rules at Gardens by the Bay events",
        "expected_venue": "GBTB-Venue-Guide",
        "type": "hybrid",
        "description": "Venue + specific restriction"
    },
    {
        "query": "Parking and transportation access to Marina Bay Sands",
        "expected_venue": "MBS-Event-Policy",
        "type": "hybrid",
        "description": "Venue + logistics"
    },
    {
        "query": "Amplified music restrictions near MRT Bayfront",
        "expected_venue": "GBTB-Venue-Guide",
        "type": "hybrid",
        "description": "MRT code + sound policy"
    }
]

print(f"📊 Evaluation dataset created: {len(test_queries)} queries")
print(f"   - Exact-match: {len([q for q in test_queries if q['type'] == 'exact_match'])} queries")
print(f"   - Semantic: {len([q for q in test_queries if q['type'] == 'semantic'])} queries")
print(f"   - Hybrid: {len([q for q in test_queries if q['type'] == 'hybrid'])} queries")

📊 Evaluation dataset created: 20 queries
   - Exact-match: 5 queries
   - Semantic: 5 queries
   - Hybrid: 10 queries


In [6]:
# A/B Evaluation Framework: Hybrid vs Hybrid + Reranking

import time
from typing import List, Dict

def evaluate_retriever(retriever, queries: List[Dict], retriever_name: str) -> Dict:
    """
    Evaluate a retriever on test queries and calculate metrics.
    
    Metrics:
    - Precision@3: Is the expected venue in top 3 results?
    - MRR (Mean Reciprocal Rank): 1/rank of first correct result
    - Avg Latency: Average query time in milliseconds
    """
    results = []
    total_latency = 0
    correct_at_3 = 0
    mrr_sum = 0
    
    print(f"\n{'='*80}")
    print(f"Evaluating: {retriever_name}")
    print(f"{'='*80}\n")
    
    for idx, test in enumerate(queries, 1):
        query = test['query']
        expected = test['expected_venue']
        query_type = test['type']
        
        # Measure latency
        start_time = time.time()
        docs = retriever.invoke(query)
        latency_ms = (time.time() - start_time) * 1000
        total_latency += latency_ms
        
        # Extract source filenames from results
        sources = []
        for doc in docs[:3]:  # Top 3 results
            source = doc.metadata.get('source', '')
            filename = source.split('\\')[-1].split('/')[-1].replace('.pdf', '')
            sources.append(filename)
        
        # Check if expected venue is in top 3
        is_correct = expected in sources
        if is_correct:
            correct_at_3 += 1
            # Calculate reciprocal rank
            rank = sources.index(expected) + 1
            mrr_sum += 1.0 / rank
        
        # Store result
        results.append({
            'query': query,
            'expected': expected,
            'type': query_type,
            'top_3_sources': sources,
            'correct': is_correct,
            'latency_ms': latency_ms
        })
        
        # Print progress
        status = "✅" if is_correct else "❌"
        print(f"{status} [{idx:2d}/{len(queries)}] {query_type:12s} | {query[:50]:50s} | {latency_ms:6.1f}ms")
    
    # Calculate aggregate metrics
    precision_at_3 = correct_at_3 / len(queries)
    mrr = mrr_sum / len(queries)
    avg_latency = total_latency / len(queries)
    
    print(f"\n{'─'*80}")
    print(f"📊 {retriever_name} Results:")
    print(f"   Precision@3: {precision_at_3:.1%} ({correct_at_3}/{len(queries)} correct)")
    print(f"   MRR: {mrr:.3f}")
    print(f"   Avg Latency: {avg_latency:.1f}ms")
    print(f"{'─'*80}\n")
    
    return {
        'retriever_name': retriever_name,
        'precision_at_3': precision_at_3,
        'mrr': mrr,
        'avg_latency_ms': avg_latency,
        'correct_count': correct_at_3,
        'total_queries': len(queries),
        'detailed_results': results
    }

print("✅ Evaluation framework ready!")

✅ Evaluation framework ready!


In [7]:
# Run A/B Comparison: Hybrid vs Hybrid + Reranking

print("🚀 Starting A/B Evaluation...")
print(f"Testing {len(test_queries)} queries on 2 retrieval systems\n")

# Evaluate baseline: Hybrid RAG only (BM25 + Semantic)
baseline_results = evaluate_retriever(hybrid_retriever, test_queries, "Hybrid RAG (BM25 + Semantic)")

# Evaluate enhanced: Hybrid RAG + Jina Reranking
rerank_results = evaluate_retriever(reranking_retriever, test_queries, "Hybrid RAG + Jina Rerank")

# Calculate improvements
precision_improvement = (rerank_results['precision_at_3'] - baseline_results['precision_at_3']) / baseline_results['precision_at_3'] * 100
mrr_improvement = (rerank_results['mrr'] - baseline_results['mrr']) / baseline_results['mrr'] * 100
latency_increase = rerank_results['avg_latency_ms'] - baseline_results['avg_latency_ms']

# Display comparison table
print("\n" + "="*80)
print("📊 FINAL COMPARISON: Hybrid RAG vs Hybrid RAG + Reranking")
print("="*80)
print()
print(f"{'Metric':<25} {'Hybrid Only':>15} {'Hybrid + Rerank':>18} {'Improvement':>15}")
print("-"*80)
print(f"{'Precision@3':<25} {baseline_results['precision_at_3']:>14.1%} {rerank_results['precision_at_3']:>18.1%} {precision_improvement:>+14.1f}%")
print(f"{'MRR':<25} {baseline_results['mrr']:>14.3f} {rerank_results['mrr']:>18.3f} {mrr_improvement:>+14.1f}%")
print(f"{'Avg Latency (ms)':<25} {baseline_results['avg_latency_ms']:>14.1f} {rerank_results['avg_latency_ms']:>18.1f} {latency_increase:>+14.1f}")
print(f"{'Correct / Total':<25} {baseline_results['correct_count']:>3d} / {baseline_results['total_queries']:<10d} {rerank_results['correct_count']:>3d} / {rerank_results['total_queries']:<13d}")
print("-"*80)

# Cost analysis (Jina Reranker pricing: ~$0.002 per query)
jina_cost_per_query = 0.002  # Approximate cost per query
total_cost_for_eval = len(test_queries) * jina_cost_per_query

print(f"\n💰 Cost Analysis:")
print(f"   Jina Reranker Cost: ${jina_cost_per_query:.4f} per query (approximate)")
print(f"   Total cost for this evaluation ({len(test_queries)} queries): ${total_cost_for_eval:.3f}")
print(f"   Free tier: 1M tokens/month on Jina AI")
print()
print("="*80)
print("✅ Evaluation complete!")
print("="*80)

🚀 Starting A/B Evaluation...
Testing 20 queries on 2 retrieval systems


Evaluating: Hybrid RAG (BM25 + Semantic)

❌ [ 1/20] exact_match  | What are the policies at postal code 018956?       | 1309.6ms
❌ [ 2/20] exact_match  | Venues accessible via MRT CE1                      |  632.3ms
❌ [ 3/20] exact_match  | What venue is located at postal code 018953?       | 1135.8ms
✅ [ 4/20] exact_match  | Marina Bay Sands event rules                       |  364.3ms
✅ [ 5/20] exact_match  | Esplanade stage specifications                     |  566.8ms
❌ [ 6/20] semantic     | What accessibility features are available for disa | 1158.6ms
✅ [ 7/20] semantic     | What are the insurance requirements for events?    |  379.3ms
✅ [ 8/20] semantic     | Rules about environmental protection at outdoor ve |  620.3ms
✅ [ 9/20] semantic     | Fire safety and emergency procedures               |  423.6ms
✅ [10/20] semantic     | Lighting and sound system capabilities             |  360.5ms
✅ [11/20] hybri

In [8]:
# Performance Breakdown by Query Type

def analyze_by_type(results_dict: Dict, type_name: str) -> Dict:
    """Calculate metrics for a specific query type."""
    detailed = results_dict['detailed_results']
    type_results = [r for r in detailed if r['type'] == type_name]
    
    if not type_results:
        return {'precision': 0, 'count': 0, 'total': 0}
    
    correct = sum(1 for r in type_results if r['correct'])
    total = len(type_results)
    precision = correct / total
    
    return {
        'precision': precision,
        'correct': correct,
        'total': total
    }

print("\n" + "="*80)
print("📊 PERFORMANCE BY QUERY TYPE")
print("="*80)
print()

for query_type in ['exact_match', 'semantic', 'hybrid']:
    baseline_type = analyze_by_type(baseline_results, query_type)
    rerank_type = analyze_by_type(rerank_results, query_type)
    
    improvement = 0
    if baseline_type['precision'] > 0:
        improvement = (rerank_type['precision'] - baseline_type['precision']) / baseline_type['precision'] * 100
    
    print(f"{'='*80}")
    print(f"{query_type.upper().replace('_', ' ')} QUERIES ({baseline_type['total']} queries)")
    print(f"{'='*80}")
    print(f"  Hybrid Only:       {baseline_type['precision']:>6.1%} ({baseline_type['correct']}/{baseline_type['total']} correct)")
    print(f"  Hybrid + Rerank:   {rerank_type['precision']:>6.1%} ({rerank_type['correct']}/{rerank_type['total']} correct)")
    print(f"  Improvement:       {improvement:>+6.1f}%")
    print()

print("="*80)
print("🎯 Key Insights:")
print("="*80)
print("Reranking adds the most value when:")
print("  1. Multiple relevant documents need fine-grained relevance scoring")
print("  2. Query combines exact terms (venue names) with concepts (policies)")
print("  3. Cross-encoder can capture query-document interactions better than embeddings")
print()
print("="*80)


📊 PERFORMANCE BY QUERY TYPE

EXACT MATCH QUERIES (5 queries)
  Hybrid Only:        40.0% (2/5 correct)
  Hybrid + Rerank:    80.0% (4/5 correct)
  Improvement:       +100.0%

SEMANTIC QUERIES (5 queries)
  Hybrid Only:        80.0% (4/5 correct)
  Hybrid + Rerank:    80.0% (4/5 correct)
  Improvement:         +0.0%

HYBRID QUERIES (10 queries)
  Hybrid Only:       100.0% (10/10 correct)
  Hybrid + Rerank:   100.0% (10/10 correct)
  Improvement:         +0.0%

🎯 Key Insights:
Reranking adds the most value when:
  1. Multiple relevant documents need fine-grained relevance scoring
  2. Query combines exact terms (venue names) with concepts (policies)
  3. Cross-encoder can capture query-document interactions better than embeddings



In [17]:
chat_loop()

Chat started! Type 'quit' to exit.

💡 Conversation history is preserved across messages within this session.

You: Can I bring a tripod to Marina Bay Sands?
AI:


At Marina Bay Sands, professional photographic equipment, including tripods, is strictly prohibited, especially at the SkyPark Observation Deck. This policy extends to any form of tripod, including table pods, flexible tripods, suction cup holders, and camera clamps. The restrictions are primarily due to safety concerns, such as strong winds that could topple the equipment and potentially injure other visitors. Large equipment could also be obstructive in crowded areas. 

For more details, you can refer to the specific policy outlined in [MBS-Event-Policy].

You: What are the sound restrictions at Gardens by the Bay?
AI:


Gardens by the Bay enforces strict sound level controls to maintain a peaceful garden atmosphere and protect both visitors and neighboring residents. Below are the sound level restrictions:

- **Daytime (9:00 AM - 6:00 PM):** Maximum of 85 dBA at venue boundaries.
- **Evening (6:00 PM - 10:00 PM):** Maximum of 80 dBA at venue boundaries.
- **After 10:00 PM:** Sound is limited to 75 dBA, applicable only for special events.

Additional amplified sound restrictions include:
- No amplified sounds or music between 10:00 PM and 9:00 AM.
- Public announcement systems can only be used for announcements.
- Loud hailers require prior approval from Gardens management.
- Background music is allowed at a conversation level (65-70 dBA).

These policies are designed to protect the serene environment of the gardens. For more details, refer to [GBTB-Venue-Guide].

You: Wheelchair accessibility at Esplanade?
AI:


The Esplanade provides excellent wheelchair accessibility features across all its performance venues. Here's what is available:

- **Accessibility Seating:** Wheelchair-accessible spaces are available in all venues, ensuring that patrons with mobility needs can enjoy events comfortably.
- **Companion Seating:** Seats are available adjacent to accessibility spaces, allowing companions to sit next to wheelchair users.
- **Aisle Seats:** These are available for patrons who may require easier access due to mobility considerations.
- **Transportation Access:** Esplanade MRT (CC3) offers a direct underground connection, and accessible parking is available at Esplanade Mall.

For any additional assistance or specific requirements, you can reach out to their Accessibility Services at accessibility@esplanade.com.

Goodbye!


### Reference (legacy)

In [ ]:
# Conversational interface with limited memory
# Document-based Question Answering using RAG

chat_loop()

Chat started! Type 'quit' to exit.



You:  How should NUS staff handle gifts?


AI: 


NUS staff must adhere to specific policies when handling gifts. The acceptance and provision of gifts, meals, and hospitality are allowed only in accordance with NUS policy documents, which include:

1. **OFN Policy on Acceptance of Gifts and Hospitality by Staff**: This policy outlines the rules for accepting gifts and hospitality to avoid conflicts of interest and maintain integrity.

2. **Sponsorship by Industry Policy**: This policy governs how sponsorships should be handled, ensuring transparency and fairness.

3. **Business Meals and Employee-Related Functions Policy**: This covers the acceptable conduct regarding business meals and related activities.

Staff should not derive any personal gain beyond their salary and employment terms from any business undertaken on behalf of the University. Compliance with these policies ensures ethical standards and helps in maintaining the integrity of the University's operations.

You:  What about if it is from a student?


...

You:  And what if the staff is also in a personal relationship with the student?


AI: 


In situations where a staff member has a personal relationship with a student, the handling of gifts becomes even more sensitive. Here are some guidelines that NUS staff should follow in such cases:

1. **Disclosure**: The staff member should disclose the personal relationship to their supervisor or relevant university office to maintain transparency and avoid any potential conflicts of interest.

2. **Avoid Conflicts of Interest**: The staff member should ensure that their personal relationship does not influence their professional responsibilities. They should recuse themselves from any academic or administrative decisions involving the student.

3. **Gifts Policy Adherence**: The acceptance of gifts should still adhere to the university's policies. If a gift is exchanged as part of the personal relationship, it should not be related to the staff member’s professional role or duties within the university.

4. **Professional Boundaries**: It is crucial for staff to maintain professional boundaries and ensure that the personal relationship does not affect their professional responsibilities or the student’s academic progress.

5. **Consultation and Guidance**: Staff members should seek guidance from their department or the appropriate university office if they are unsure of how to navigate the situation while complying with university policies.

By adhering to these principles, staff members can manage the potential risks associated with personal relationships with students and maintain ethical standards.

You:  quit


Goodbye!


In [ ]:
# Text-to-image generation with prompt engineering

chat_loop()

Chat started! Type 'quit' to exit.



You:  Image of a white siamese cat


AI:


Here is an image of a white Siamese cat:

![White Siamese Cat](https://replicate.delivery/xezq/13xSdaao6WqSDxIeqJX0sRS1rUEKWPYpXQ06SKwxjPP6CTwKA/tmpu2shrd04.webp)

You:  Now show the results from these improved prompts: "a white siamese cat, studio lighting",  "a white siamese cat, dramatic lighting, dark background",  "a white siamese cat, outdoor setting, natural light"


AI:


Here are the images of a white Siamese cat with different lighting and settings:

1. **Studio Lighting**:
   ![White Siamese Cat, Studio Lighting](https://replicate.delivery/xezq/UUyadShxv3YBDFakR3jm3Da1vQsef1Kr7FcjOCqqfCkdMMBrA/tmpkyrjatek.webp)

2. **Dramatic Lighting, Dark Background**:
   ![White Siamese Cat, Dramatic Lighting, Dark Background](https://replicate.delivery/xezq/rBqJY6rLqbJPLZI6vSWCUuYy3648E0PtbftLYlr9ExwJDTwKA/tmpdkht_jw9.webp)

3. **Outdoor Setting, Natural Light**:
   ![White Siamese Cat, Outdoor Setting, Natural Light](https://replicate.delivery/xezq/p3QYKt4jorIjClVqUOeZAFJrmiFkewgavQwCAzPuHfOwMMBrA/tmpa7ag8qtx.webp)

You:  quit


Goodbye!


In [ ]:
# Multi-agent task handling using a controller (Weather, SQL, Recommender)

chat_loop()

Chat started! Type 'quit' to exit.



You:  Recommend some events happening today


AI:


Here are some event recommendations for today in Singapore:

1. **Orchard Mall Art Fair**:
   - **Type**: Indoor
   - **Description**: Pop-up art and design exhibition.
   - **Location**: ION Orchard, Singapore
   - **Why Attend**: With the high temperature and humidity making it feel like 39.7°C (103.5°F), an indoor event like this is a comfortable choice. You can enjoy the art and design exhibition in a climate-controlled environment, avoiding the heat and humidity outside.

2. **Marina Bay Food Festival**:
   - **Type**: Outdoor
   - **Description**: A celebration of local and international cuisine.
   - **Location**: Marina Bay, Singapore
   - **Why Attend**: Although it's an outdoor event, the partly cloudy skies and low UV index mean you won't be exposed to intense sunlight, and the slight breeze might offer some relief. However, be prepared for the warm and humid conditions.

Both events offer unique experiences, so consider your comfort with the weather when making your choice.

You:  What about outdoor events in India?


...

You:  Would you recommend this based on the weather in Mumbai?


AI:


The current weather in Mumbai, India, is misty with a temperature of 34.2°C (93.6°F), but it feels like 42.9°C (109.2°F) due to the humidity. The UV index is moderate at 3.3, and there is a light breeze from the west at 7.4 mph (11.9 kph).

Given these conditions, attending the "Mumbai Music Street" outdoor event at Marine Drive could be enjoyable if you are prepared for the warm and humid weather. Make sure to stay hydrated, wear light clothing, and perhaps bring a fan or umbrella for shade. The misty conditions might add a unique atmosphere to the event, but visibility is somewhat limited at 3.0 km.

You:  quit


Goodbye!
